In [ ]:
# path to our tables in the silver layer
silver_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/4e18f678-60fe-4988-82c1-2d51453db35e/Tables"
# path to our tables in the gold layer
gold_tables_path = "abfss://3201caea-2820-458c-8915-9fe2f28ba2a7@onelake.dfs.fabric.microsoft.com/63f7f215-36ce-4efd-b681-163d607668af/Tables"

StatementMeta(, , , Waiting, )

In [ ]:
df_product_silver = spark.read.format("delta").load(f"{silver_tables_path}/Product")
df_region_silver = spark.read.format("delta").load(f"{silver_tables_path}/Region")
df_reseller_silver = spark.read.format("delta").load(f"{silver_tables_path}/Reseller")
df_sales_silver = spark.read.format("delta").load(f"{silver_tables_path}/Sales")
df_salespersonregion_silver = spark.read.format("delta").load(f"{silver_tables_path}/SalesPersonRegion")
df_salesperson_silver = spark.read.format("delta").load(f"{silver_tables_path}/Salesperson")

StatementMeta(, , , Waiting, )

In [ ]:
from pyspark.sql.functions import col
# Only show active, remove startDate and endDate, hashdiff
columns_to_remove = ["isActive", "startDate", "endDate", "hashdiff"]
df_salesperson_silver = df_salesperson_silver.filter(col("isActive") == True)
df_salesperson_silver = df_salesperson_silver.drop(*columns_to_remove)

StatementMeta(, , , Waiting, )

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id

df_salespersonregion_silver = df_salespersonregion_silver.withColumn("EmployeeRegionKey", monotonically_increasing_id())

df_salespersonregion_silver = df_salespersonregion_silver.select(
    "EmployeeRegionKey",
    "SalesTerritoryKey",
    "EmployeeKey"
)

StatementMeta(, , , Waiting, )

In [ ]:
from pyspark.sql.functions import to_date, year, month, dayofmonth

df_date = df_sales_silver.select(to_date("OrderDate").alias("Date")).distinct()

df_date = df_date.withColumn("year", year("Date")) \
                  .withColumn("month", month("Date")) \
                  .withColumn("day", dayofmonth("Date"))

display(df_date)

StatementMeta(, , , Waiting, )

SynapseWidget(Synapse.DataFrame, 77f75731-929e-41a0-899e-f93701c27801)

In [ ]:
df_product_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Dim_Product")
df_region_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Dim_Region")
df_reseller_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Dim_Reseller")
df_salesperson_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Dim_Employee")
df_salespersonregion_silver.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Bridge_EmployeeRegion")
df_date.write.format("delta").mode("overwrite").save(f"{gold_tables_path}/Dim_Date")

StatementMeta(, , , Waiting, )